In [1]:
%load_ext snakeviz
%config InlineBackend.figure_format = 'svg'
%config InlineBackend.figure_format = 'png'

In [2]:
from email import generator
import numpy as np
from python.ADMPForce import ADMPGenerator
import scipy
from scipy.stats import special_ortho_group
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *
import mpidplugin

mScales = np.array([0.0, 0.0, 0.0, 1.0])
pScales = np.array([0.0, 0.0, 0.0, 1.0])
dScales = np.array([0.0, 0.0, 0.0, 1.0])
rc = 8 # in Angstrom
ethresh = 1e-4


pdb = 'tests/samples/waterdimer_aligned.pdb'
xml = 'tests/samples/mpidwater.xml'
generator = ADMPGenerator(pdb, xml, rc, ethresh, mScales, pScales, dScales, )
# get a random geometry for testing
scipy.random.seed(1000)
R1 = special_ortho_group.rvs(3)
R2 = special_ortho_group.rvs(3)

positions = generator.positions
positions[0:3] = positions[0:3].dot(R1)
positions[3:6] = positions[3:6].dot(R2)
positions[3:] += np.array([3.0, 0.0, 0.0])


force = generator.create_force()
force.update()
force.kappa = 0.328532611


In [3]:
print(positions)

[[ 0.          0.          0.        ]
 [-0.72004306 -0.01869945  0.63058173]
 [ 0.68557522  0.50825211  0.43371812]
 [ 3.          0.          0.        ]
 [ 3.02161682 -0.52203431 -0.80215827]
 [ 2.08174622 -0.01347945  0.27032073]]


In [10]:
%timeit ene0 = force.calc_reci_space_energy()
%timeit force0 = force.calc_reci_space_force()
print(force0)

In [9]:
from python.pme import pme_reciprocal, pme_reciprocal_force

%timeit pme_reciprocal_force(force.positions, force.box, force.Q, force.kappa, force.lmax, force.K1, force.K2, force.K3)

10.5 ms ± 933 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
